In [1]:
import sqlite3
import json
from datetime import datetime

timeframe = '2017-10'
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
cursor = connection.cursor()

In [2]:
def create_table():
    cursor.execute('''CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT,
    comment TEXT, subreddit TEXT, unix INT, score INT)''')

In [3]:
create_table()

In [4]:
def format_data(data):
    data = data.replace('\n', ' newlinechar ').replace('\r', ' newlinechar ').replace('"', "'")
    return data

In [5]:
def find_parent(pid):
    try:
        sql = "SELECT comment from parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        cursor.execute(sql)
        result = cursor.fetchone()

        if result != None:
            return result[0]
        else:
            return False
        
    except Exception as e:
        print('find_parent', e)
        return False

In [6]:
def find_existing_score(pid):
    try:
        sql = "SELECT score from parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        cursor.execute(sql)
        result = cursor.fetchone()

        if result != None:
            return result[0]
        else:
            return False
        
    except Exception as e:
        print('find_parent', e)
        return False

In [7]:
def is_acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True

In [8]:
def transaction_builder(sql):
    global sql_transaction
    sql_transaction.append(sql)
    
    if len(sql_transaction) > 1000:
        cursor.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                cursor.execute(s)
            except:
                pass
            
        connection.commit()
        sql_transaction = []

        
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = '''UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;'''.format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_builder(sql)
    except Exception as e:
        print('replace_comment',str(e))

        
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = '''INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});'''.format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_builder(sql)
    except Exception as e:
        print('has_parent', e)


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = '''INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});'''.format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_builder(sql)
    except Exception as e:
        print('no_parent', e)   

In [9]:
row_counter = 0
paired_rows = 0

START_ROW = 0
CLEANUP = 1000000

SCORE_THRESHOLD = 2

with open('D:\AI_Data\{}\RC_{}'.format(timeframe.split('-')[0], timeframe), buffering=1000) as f:
    for row in f:
        row_counter += 1
        
        if row_counter > START_ROW:
            try:
                row = json.loads(row)
                parent_id = row['parent_id'].split('_')[1]
                body = format_data(row['body'])
                created_utc = row['created_utc']
                score = row['score']
                
                comment_id = row['id']
                
                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)
                
                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        if is_acceptable(body):
                            sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            
                else:
                    if is_acceptable(body):
                        if parent_data:
                            if score >= 2:
                                sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
            except Exception as e:
                print(str(e))
                            
        if row_counter % 100000 == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

        if row_counter > START_ROW:
            if row_counter % CLEANUP == 0:
                print("Cleanin up!")
                sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                cursor.execute(sql)
                connection.commit()
                cursor.execute("VACUUM")
                connection.commit()

Total Rows Read: 100000, Paired Rows: 4610, Time: 2017-12-07 20:21:58.842129
Total Rows Read: 200000, Paired Rows: 10867, Time: 2017-12-07 20:22:50.875467
Total Rows Read: 300000, Paired Rows: 17417, Time: 2017-12-07 20:23:41.233623
Total Rows Read: 400000, Paired Rows: 24389, Time: 2017-12-07 20:24:31.893815
Total Rows Read: 500000, Paired Rows: 31402, Time: 2017-12-07 20:25:27.988792
Total Rows Read: 600000, Paired Rows: 38542, Time: 2017-12-07 20:26:21.570199
Total Rows Read: 700000, Paired Rows: 45804, Time: 2017-12-07 20:27:16.622667
Total Rows Read: 800000, Paired Rows: 52551, Time: 2017-12-07 20:28:08.841375
Total Rows Read: 900000, Paired Rows: 59594, Time: 2017-12-07 20:29:02.293753
Total Rows Read: 1000000, Paired Rows: 66822, Time: 2017-12-07 20:29:54.821449
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 72669, Time: 2017-12-07 20:30:46.937871
Total Rows Read: 1200000, Paired Rows: 79636, Time: 2017-12-07 20:31:30.169006
Total Rows Read: 1300000, Paired Rows: 86985, Time

Total Rows Read: 10300000, Paired Rows: 761565, Time: 2017-12-07 21:52:29.679928
Total Rows Read: 10400000, Paired Rows: 769834, Time: 2017-12-07 21:53:21.564731
Total Rows Read: 10500000, Paired Rows: 778200, Time: 2017-12-07 21:54:18.255027
Total Rows Read: 10600000, Paired Rows: 786682, Time: 2017-12-07 21:55:19.420092
Total Rows Read: 10700000, Paired Rows: 794995, Time: 2017-12-07 21:56:12.479383
Total Rows Read: 10800000, Paired Rows: 803293, Time: 2017-12-07 21:57:09.599193
Total Rows Read: 10900000, Paired Rows: 811497, Time: 2017-12-07 21:58:01.137401
Total Rows Read: 11000000, Paired Rows: 819502, Time: 2017-12-07 21:59:01.746475
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 825334, Time: 2017-12-07 22:00:09.586480
Total Rows Read: 11200000, Paired Rows: 832563, Time: 2017-12-07 22:00:55.897862
Total Rows Read: 11300000, Paired Rows: 840203, Time: 2017-12-07 22:01:42.632313
Total Rows Read: 11400000, Paired Rows: 847605, Time: 2017-12-07 22:02:30.042530
Total Rows Read:

Total Rows Read: 20200000, Paired Rows: 1506187, Time: 2017-12-07 23:14:19.665399
Total Rows Read: 20300000, Paired Rows: 1513460, Time: 2017-12-07 23:15:05.581509
Total Rows Read: 20400000, Paired Rows: 1521137, Time: 2017-12-07 23:15:51.627772
Total Rows Read: 20500000, Paired Rows: 1529258, Time: 2017-12-07 23:16:43.408452
Total Rows Read: 20600000, Paired Rows: 1537609, Time: 2017-12-07 23:17:34.122663
Total Rows Read: 20700000, Paired Rows: 1545525, Time: 2017-12-07 23:18:23.440711
Total Rows Read: 20800000, Paired Rows: 1553016, Time: 2017-12-07 23:19:10.410537
Total Rows Read: 20900000, Paired Rows: 1560513, Time: 2017-12-07 23:19:57.274547
Total Rows Read: 21000000, Paired Rows: 1567875, Time: 2017-12-07 23:20:41.670405
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 1573204, Time: 2017-12-07 23:20:49.145990
Total Rows Read: 21200000, Paired Rows: 1579943, Time: 2017-12-07 23:21:34.451055
Total Rows Read: 21300000, Paired Rows: 1587163, Time: 2017-12-07 23:22:19.906353
Tota

'>=' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>=' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>=' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances o

'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
Total Rows Read: 24800000, Paired Rows: 1846758, Time: 2017-12-07 23:52:49.507955
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
'>' not supported between instances of 'NoneType' and 'int'
Total Rows Read: 2

...

Total Rows Read: 40500000, Paired Rows: 3009686, Time: 2017-12-08 02:25:25.187134
Total Rows Read: 40600000, Paired Rows: 3017283, Time: 2017-12-08 02:26:11.184984
Total Rows Read: 40700000, Paired Rows: 3025062, Time: 2017-12-08 02:26:57.520629
Total Rows Read: 40800000, Paired Rows: 3033070, Time: 2017-12-08 02:27:44.348740
Total Rows Read: 40900000, Paired Rows: 3040872, Time: 2017-12-08 02:28:30.152139
Total Rows Read: 41000000, Paired Rows: 3049041, Time: 2017-12-08 02:29:16.297132
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 3055211, Time: 2017-12-08 02:31:18.058906
Total Rows Read: 41200000, Paired Rows: 3062316, Time: 2017-12-08 02:32:03.680800
Total Rows Read: 41300000, Paired Rows: 3069815, Time: 2017-12-08 02:32:50.213097
Total Rows Read: 41400000, Paired Rows: 3077159, Time: 2017-12-08 02:33:36.273503
Total Rows Read: 41500000, Paired Rows: 3084719, Time: 2017-12-08 02:34:22.165266
Total Rows Read: 41600000, Paired Rows: 3092623, Time: 2017-12-08 02:35:08.555464
Tota

Total Rows Read: 50200000, Paired Rows: 3741619, Time: 2017-12-08 03:55:11.845302
Total Rows Read: 50300000, Paired Rows: 3748911, Time: 2017-12-08 03:55:57.983971
Total Rows Read: 50400000, Paired Rows: 3756069, Time: 2017-12-08 03:56:44.767993
Total Rows Read: 50500000, Paired Rows: 3763005, Time: 2017-12-08 03:57:31.266834
Total Rows Read: 50600000, Paired Rows: 3769782, Time: 2017-12-08 03:58:17.264334
Total Rows Read: 50700000, Paired Rows: 3777219, Time: 2017-12-08 03:59:04.126687
Total Rows Read: 50800000, Paired Rows: 3785094, Time: 2017-12-08 03:59:49.686236
Total Rows Read: 50900000, Paired Rows: 3793279, Time: 2017-12-08 04:00:35.657250
Total Rows Read: 51000000, Paired Rows: 3801422, Time: 2017-12-08 04:01:21.326758
Cleanin up!
Total Rows Read: 51100000, Paired Rows: 3807387, Time: 2017-12-08 04:03:48.353374
Total Rows Read: 51200000, Paired Rows: 3814980, Time: 2017-12-08 04:04:34.706791
Total Rows Read: 51300000, Paired Rows: 3823030, Time: 2017-12-08 04:05:21.112786
Tota

Total Rows Read: 59800000, Paired Rows: 4463461, Time: 2017-12-08 05:24:55.752919
Total Rows Read: 59900000, Paired Rows: 4470659, Time: 2017-12-08 05:25:39.974358
Total Rows Read: 60000000, Paired Rows: 4478319, Time: 2017-12-08 05:26:23.869362
Cleanin up!
Total Rows Read: 60100000, Paired Rows: 4484079, Time: 2017-12-08 05:29:01.968451
Total Rows Read: 60200000, Paired Rows: 4490982, Time: 2017-12-08 05:29:47.182687
Total Rows Read: 60300000, Paired Rows: 4497830, Time: 2017-12-08 05:30:32.242986
Total Rows Read: 60400000, Paired Rows: 4504368, Time: 2017-12-08 05:31:16.516785
Total Rows Read: 60500000, Paired Rows: 4511675, Time: 2017-12-08 05:32:01.305776
Total Rows Read: 60600000, Paired Rows: 4518921, Time: 2017-12-08 05:32:46.048433
Total Rows Read: 60700000, Paired Rows: 4525669, Time: 2017-12-08 05:33:29.691745
Total Rows Read: 60800000, Paired Rows: 4532553, Time: 2017-12-08 05:34:13.258757
Total Rows Read: 60900000, Paired Rows: 4539291, Time: 2017-12-08 05:34:56.916311
Tota

Total Rows Read: 69700000, Paired Rows: 5209363, Time: 2017-12-08 07:03:29.189101
Total Rows Read: 69800000, Paired Rows: 5216823, Time: 2017-12-08 07:04:15.704181
Total Rows Read: 69900000, Paired Rows: 5223761, Time: 2017-12-08 07:05:01.660554
Total Rows Read: 70000000, Paired Rows: 5230434, Time: 2017-12-08 07:05:47.488163
Cleanin up!
Total Rows Read: 70100000, Paired Rows: 5236053, Time: 2017-12-08 07:09:01.120692
Total Rows Read: 70200000, Paired Rows: 5242532, Time: 2017-12-08 07:09:49.501715
Total Rows Read: 70300000, Paired Rows: 5249171, Time: 2017-12-08 07:10:38.045551
Total Rows Read: 70400000, Paired Rows: 5256234, Time: 2017-12-08 07:11:26.256057
Total Rows Read: 70500000, Paired Rows: 5263835, Time: 2017-12-08 07:12:14.246880
Total Rows Read: 70600000, Paired Rows: 5271687, Time: 2017-12-08 07:13:02.129886
Total Rows Read: 70700000, Paired Rows: 5279930, Time: 2017-12-08 07:13:50.471319
Total Rows Read: 70800000, Paired Rows: 5288250, Time: 2017-12-08 07:14:38.453415
Tota

Total Rows Read: 79400000, Paired Rows: 5938142, Time: 2017-12-08 08:45:54.352750
Total Rows Read: 79500000, Paired Rows: 5945262, Time: 2017-12-08 08:46:41.386505
Total Rows Read: 79600000, Paired Rows: 5953149, Time: 2017-12-08 08:47:28.311253
Total Rows Read: 79700000, Paired Rows: 5961238, Time: 2017-12-08 08:48:16.148794
Total Rows Read: 79800000, Paired Rows: 5969314, Time: 2017-12-08 08:49:03.596979
Total Rows Read: 79900000, Paired Rows: 5977064, Time: 2017-12-08 08:49:51.209787
Total Rows Read: 80000000, Paired Rows: 5984734, Time: 2017-12-08 08:50:37.489386
Cleanin up!
Total Rows Read: 80100000, Paired Rows: 5990968, Time: 2017-12-08 08:54:12.375927
Total Rows Read: 80200000, Paired Rows: 5998107, Time: 2017-12-08 08:55:00.175266
Total Rows Read: 80300000, Paired Rows: 6005456, Time: 2017-12-08 08:55:47.862282
Total Rows Read: 80400000, Paired Rows: 6012722, Time: 2017-12-08 08:56:35.871090
Total Rows Read: 80500000, Paired Rows: 6019845, Time: 2017-12-08 08:57:22.879388
Tota